referensi https://www.analyticsvidhya.com/blog/2021/04/forward-feature-selection-and-its-implementation/

In [ ]:
#instal library mlxtend
# %pip install mlxtend

In [1]:
#libraries buat seleksi fitur
import pandas as pd
import numpy as np
from mlxtend.feature_selection import SequentialFeatureSelector as sfs
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import f_classif,SelectKBest 
from sklearn.feature_selection import mutual_info_classif

#libraries buat klasifikasi
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_validate
from sklearn import metrics
from sklearn.ensemble import RandomForestClassifier
from sklearn import svm

In [2]:
#import dataset
df = pd.read_csv('nirsMangga.csv')

# separate dataset into train and test
x_train, x_test, y_train, y_test = train_test_split(
    df.drop(labels=['No', 'Mango Cultivars', 'Vit C (mg/100g)', 'TA (mg/100g)', 'SSC (oBrix)', 'label'], axis=1),
    df['label'],
    test_size=0.3,
    random_state=0)

x_train.head()

,999.9,1000.3,1000.7,1001.1,1001.4,1001.8,1002.2,1002.6,1003,1003.4,...,2478.7,2481.1,2483.5,2485.8,2488.2,2490.6,2493,2495.4,2497.8,2500.2
16,0.471459,0.471074,0.470934,0.470379,0.470260,0.469880,0.469497,0.469435,0.469454,0.468998,...,1.413537,1.415740,1.417568,1.419698,1.421711,1.423070,1.424394,1.426121,1.427552,1.428625
51,0.433239,0.432622,0.432626,0.432379,0.431620,0.430710,0.430836,0.430847,0.430188,0.429470,...,1.601232,1.602877,1.604524,1.605982,1.606778,1.607837,1.608756,1.609967,1.610900,1.611099
183,0.545045,0.544204,0.543792,0.543596,0.543338,0.542534,0.541493,0.541139,0.541308,0.540831,...,1.524657,1.525973,1.527454,1.529518,1.530097,1.530315,1.530254,1.531191,1.532366,1.533183
145,0.545846,0.544815,0.544524,0.544631,0.544169,0.543143,0.542535,0.542080,0.541842,0.541258,...,1.421962,1.422955,1.423717,1.424639,1.425080,1.425797,1.426503,1.427164,1.427838,1.428271
40,0.381048,0.380483,0.380541,0.380151,0.379599,0.379189,0.379009,0.378722,0.378309,0.377719,...,1.571125,1.572674,1.574303,1.576075,1.577273,1.577980,1.578561,1.579334,1.580042,1.581424


In [3]:
%%time

# determine the mutual information
mutual_info = mutual_info_classif(x_train, y_train)
mutual_info

CPU times: user 4.24 s, sys: 96 ms, total: 4.34 s
Wall time: 4.33 s


array([0.63194121, 0.63677335, 0.63393088, ..., 0.5091611 , 0.49290951,
       0.48903907])

In [4]:
x_train.columns

Index(['999.9', '1000.3', '1000.7', '1001.1', '1001.4', '1001.8', '1002.2',
       '1002.6', '1003', '1003.4',
       ...
       '2478.7', '2481.1', '2483.5', '2485.8', '2488.2', '2490.6', '2493',
       '2495.4', '2497.8', '2500.2'],
      dtype='object', length=1557)

In [4]:
# Create features score, features p_value, and features name
features_mi = pd.DataFrame(mutual_info)
features = pd.DataFrame(x_train.columns)
features_mi = pd.concat([features, features_mi],axis=1)

#reset index, supaya tidak ada nilai nan di dataframe yg dibikin
features_mi.reset_index(drop=True, inplace=True)
features.reset_index(drop=True, inplace=True)

# Assign the column name
features_mi.columns = ["Input_Features", "Mutual Information Value"]

# Print features score
print(features_mi.nlargest(40,columns="Mutual Information Value"))

    Input_Features  Mutual Information Value
318         1139.7                  0.709039
319         1140.2                  0.703579
322         1141.7                  0.702486
315         1138.2                  0.700575
316         1138.7                  0.699476
317         1139.2                  0.696952
320         1140.7                  0.695759
321         1141.2                  0.689817
323         1142.2                  0.689658
303         1132.2                  0.687375
314         1137.7                  0.686203
302         1131.7                  0.681486
313         1137.2                  0.678820
686         1359.6                  0.678404
685         1358.9                  0.676511
324         1142.7                  0.673497
307         1134.2                  0.671183
687         1360.3                  0.667588
306         1133.7                  0.667333
688           1361                  0.666494
301         1131.2                  0.665180
304       

# Seleksi Fitur Backward Selection dengan Random Forest Classifier

In [5]:
x_train_selected = x_train[features_mi.nlargest(300,columns="Mutual Information Value").iloc[0:300, 0]]
x_test_selected = x_test[features_mi.nlargest(300,columns="Mutual Information Value").iloc[0:300, 0]]

In [6]:
x_train_selected

,1139.7,1140.2,1141.7,1138.2,1138.7,1139.2,1140.7,1141.2,1142.2,1132.2,...,1333,1287.4,1207.6,1346.2,1007.3,1212.7,1320.8,1610.4,1335.8,1336.5
16,0.492063,0.493875,0.500545,0.485824,0.487906,0.489890,0.496089,0.498279,0.502916,0.464507,...,0.634095,0.556184,0.569706,0.665571,0.465651,0.568512,0.603703,0.842620,0.641125,0.642758
51,0.464010,0.466825,0.475396,0.454894,0.457810,0.460916,0.469729,0.472565,0.478432,0.426489,...,0.668614,0.555730,0.573913,0.715781,0.425145,0.572004,0.623792,0.985393,0.679198,0.681701
183,0.586312,0.589698,0.600224,0.575629,0.579215,0.582786,0.593261,0.596755,0.603650,0.540704,...,0.810916,0.692120,0.707730,0.855471,0.534954,0.706122,0.766246,1.073005,0.820799,0.823288
145,0.577506,0.580543,0.589776,0.568155,0.571318,0.574446,0.583619,0.586613,0.592968,0.538299,...,0.776353,0.669018,0.687148,0.818788,0.536735,0.685297,0.734829,1.030706,0.785930,0.788264
40,0.402640,0.405209,0.413083,0.395286,0.397766,0.400044,0.407849,0.410373,0.415739,0.370542,...,0.579554,0.480875,0.503912,0.621508,0.373607,0.501926,0.539874,0.869928,0.588804,0.591019
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
103,0.543538,0.546104,0.554676,0.535210,0.538105,0.540964,0.549027,0.551875,0.557337,0.508815,...,0.730200,0.630338,0.643349,0.769883,0.504845,0.642064,0.691601,0.982645,0.739100,0.741312
67,0.473749,0.476239,0.484654,0.465488,0.468295,0.471059,0.479081,0.481982,0.487460,0.439093,...,0.661354,0.556828,0.573851,0.704487,0.439315,0.572048,0.619995,0.938691,0.671106,0.673352
117,0.503392,0.506553,0.516428,0.493610,0.496839,0.500089,0.509868,0.513116,0.519892,0.462708,...,0.740155,0.607688,0.629789,0.795169,0.461213,0.627599,0.687533,1.094887,0.752581,0.755331
47,0.459436,0.462290,0.471228,0.450392,0.453445,0.456400,0.465182,0.468146,0.474372,0.421441,...,0.665888,0.550905,0.568889,0.713774,0.420312,0.566776,0.620321,0.980873,0.676627,0.679137


In [7]:
# calling the linear regression model
lreg = RandomForestClassifier(n_estimators=100)
sfs1 = sfs(lreg, k_features=40, forward=False, verbose=2, scoring='accuracy', n_jobs=-1)

In [8]:
%%time

sfs1 = sfs1.fit(x_train_selected, y_train)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 80 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:    7.9s
[Parallel(n_jobs=-1)]: Done 292 out of 300 | elapsed:   27.2s remaining:    0.7s
[Parallel(n_jobs=-1)]: Done 300 out of 300 | elapsed:   27.5s finished

[2022-06-06 18:46:10] Features: 299/40 -- score: 0.876923076923077[Parallel(n_jobs=-1)]: Using backend LokyBackend with 80 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:    3.1s
[Parallel(n_jobs=-1)]: Done 290 out of 299 | elapsed:   20.7s remaining:    0.6s
[Parallel(n_jobs=-1)]: Done 299 out of 299 | elapsed:   21.2s finished

[2022-06-06 18:46:31] Features: 298/40 -- score: 0.876923076923077[Parallel(n_jobs=-1)]: Using backend LokyBackend with 80 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:    2.9s
[Parallel(n_jobs=-1)]: Done 289 out of 298 | elapsed:   20.5s remaining:    0.6s
[Parallel(n_jobs=-1)]: Done 298 out of 298 | elapsed:   21.0s fi

[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:    1.4s
[Parallel(n_jobs=-1)]: Done 257 out of 277 | elapsed:   17.1s remaining:    1.3s
[Parallel(n_jobs=-1)]: Done 277 out of 277 | elapsed:   17.9s finished

[2022-06-06 18:53:38] Features: 276/40 -- score: 0.876923076923077[Parallel(n_jobs=-1)]: Using backend LokyBackend with 80 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:    1.4s
[Parallel(n_jobs=-1)]: Done 256 out of 276 | elapsed:   16.6s remaining:    1.3s
[Parallel(n_jobs=-1)]: Done 276 out of 276 | elapsed:   17.4s finished

[2022-06-06 18:53:56] Features: 275/40 -- score: 0.8846153846153847[Parallel(n_jobs=-1)]: Using backend LokyBackend with 80 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:    1.4s
[Parallel(n_jobs=-1)]: Done 254 out of 275 | elapsed:   16.6s remaining:    1.4s
[Parallel(n_jobs=-1)]: Done 275 out of 275 | elapsed:   17.2s finished

[2022-06-06 18:54:13] Features: 274/40 -- score: 0.876923076923077[P

[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:    1.2s
[Parallel(n_jobs=-1)]: Done 223 out of 254 | elapsed:   12.5s remaining:    1.7s
[Parallel(n_jobs=-1)]: Done 254 out of 254 | elapsed:   13.3s finished

[2022-06-06 18:59:25] Features: 253/40 -- score: 0.876923076923077[Parallel(n_jobs=-1)]: Using backend LokyBackend with 80 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:    1.0s
[Parallel(n_jobs=-1)]: Done 221 out of 253 | elapsed:   11.4s remaining:    1.7s
[Parallel(n_jobs=-1)]: Done 253 out of 253 | elapsed:   12.5s finished

[2022-06-06 18:59:38] Features: 252/40 -- score: 0.876923076923077[Parallel(n_jobs=-1)]: Using backend LokyBackend with 80 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:    1.3s
[Parallel(n_jobs=-1)]: Done 220 out of 252 | elapsed:   11.9s remaining:    1.7s
[Parallel(n_jobs=-1)]: Done 252 out of 252 | elapsed:   13.1s finished

[2022-06-06 18:59:51] Features: 251/40 -- score: 0.876923076923077[Pa

[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:    1.1s
[Parallel(n_jobs=-1)]: Done 188 out of 231 | elapsed:   10.1s remaining:    2.3s
[Parallel(n_jobs=-1)]: Done 231 out of 231 | elapsed:   11.9s finished

[2022-06-06 19:04:05] Features: 230/40 -- score: 0.876923076923077[Parallel(n_jobs=-1)]: Using backend LokyBackend with 80 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:    1.0s
[Parallel(n_jobs=-1)]: Done 187 out of 230 | elapsed:    9.7s remaining:    2.2s
[Parallel(n_jobs=-1)]: Done 230 out of 230 | elapsed:   11.5s finished

[2022-06-06 19:04:16] Features: 229/40 -- score: 0.876923076923077[Parallel(n_jobs=-1)]: Using backend LokyBackend with 80 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:    1.1s
[Parallel(n_jobs=-1)]: Done 185 out of 229 | elapsed:    9.7s remaining:    2.3s
[Parallel(n_jobs=-1)]: Done 229 out of 229 | elapsed:   11.5s finished

[2022-06-06 19:04:28] Features: 228/40 -- score: 0.8846153846153847[P

[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:    1.0s
[Parallel(n_jobs=-1)]: Done 154 out of 208 | elapsed:    8.5s remaining:    3.0s
[Parallel(n_jobs=-1)]: Done 208 out of 208 | elapsed:   11.0s finished

[2022-06-06 19:08:56] Features: 207/40 -- score: 0.876923076923077[Parallel(n_jobs=-1)]: Using backend LokyBackend with 80 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:    1.0s
[Parallel(n_jobs=-1)]: Done 152 out of 207 | elapsed:    8.6s remaining:    3.1s
[Parallel(n_jobs=-1)]: Done 207 out of 207 | elapsed:   11.3s finished

[2022-06-06 19:09:07] Features: 206/40 -- score: 0.876923076923077[Parallel(n_jobs=-1)]: Using backend LokyBackend with 80 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:    1.0s
[Parallel(n_jobs=-1)]: Done 151 out of 206 | elapsed:    8.6s remaining:    3.1s
[Parallel(n_jobs=-1)]: Done 206 out of 206 | elapsed:   11.3s finished

[2022-06-06 19:09:18] Features: 205/40 -- score: 0.8846153846153847[P

...

[Parallel(n_jobs=-1)]: Done  32 out of  95 | elapsed:    3.1s remaining:    6.1s
[Parallel(n_jobs=-1)]: Done  80 out of  95 | elapsed:    6.6s remaining:    1.2s
[Parallel(n_jobs=-1)]: Done  95 out of  95 | elapsed:    7.7s finished

[2022-06-06 19:27:13] Features: 94/40 -- score: 0.8692307692307694[Parallel(n_jobs=-1)]: Using backend LokyBackend with 80 concurrent workers.
[Parallel(n_jobs=-1)]: Done  31 out of  94 | elapsed:    2.9s remaining:    5.8s
[Parallel(n_jobs=-1)]: Done  79 out of  94 | elapsed:    6.3s remaining:    1.2s
[Parallel(n_jobs=-1)]: Done  94 out of  94 | elapsed:    7.4s finished

[2022-06-06 19:27:21] Features: 93/40 -- score: 0.876923076923077[Parallel(n_jobs=-1)]: Using backend LokyBackend with 80 concurrent workers.
[Parallel(n_jobs=-1)]: Done  28 out of  93 | elapsed:    2.7s remaining:    6.4s
[Parallel(n_jobs=-1)]: Done  75 out of  93 | elapsed:    6.3s remaining:    1.5s
[Parallel(n_jobs=-1)]: Done  93 out of  93 | elapsed:    7.4s finished

[2022-06-06 1

[Parallel(n_jobs=-1)]: Done  62 out of  73 | elapsed:    5.4s remaining:    1.0s
[Parallel(n_jobs=-1)]: Done  73 out of  73 | elapsed:    6.1s finished

[2022-06-06 19:29:45] Features: 72/40 -- score: 0.876923076923077[Parallel(n_jobs=-1)]: Using backend LokyBackend with 80 concurrent workers.
[Parallel(n_jobs=-1)]: Done  24 out of  72 | elapsed:    2.5s remaining:    5.0s
[Parallel(n_jobs=-1)]: Done  61 out of  72 | elapsed:    5.3s remaining:    1.0s
[Parallel(n_jobs=-1)]: Done  72 out of  72 | elapsed:    6.0s finished

[2022-06-06 19:29:51] Features: 71/40 -- score: 0.876923076923077[Parallel(n_jobs=-1)]: Using backend LokyBackend with 80 concurrent workers.
[Parallel(n_jobs=-1)]: Done  20 out of  71 | elapsed:    2.2s remaining:    5.5s
[Parallel(n_jobs=-1)]: Done  56 out of  71 | elapsed:    5.0s remaining:    1.3s
[Parallel(n_jobs=-1)]: Done  71 out of  71 | elapsed:    6.2s finished

[2022-06-06 19:29:57] Features: 70/40 -- score: 0.876923076923077[Parallel(n_jobs=-1)]: Using b

[Parallel(n_jobs=-1)]: Done  21 out of  50 | elapsed:    2.2s remaining:    3.1s
[Parallel(n_jobs=-1)]: Done  47 out of  50 | elapsed:    4.3s remaining:    0.3s
[Parallel(n_jobs=-1)]: Done  50 out of  50 | elapsed:    4.5s finished

[2022-06-06 19:31:47] Features: 49/40 -- score: 0.8692307692307694[Parallel(n_jobs=-1)]: Using backend LokyBackend with 80 concurrent workers.
[Parallel(n_jobs=-1)]: Done  15 out of  49 | elapsed:    1.9s remaining:    4.3s
[Parallel(n_jobs=-1)]: Done  40 out of  49 | elapsed:    3.7s remaining:    0.8s
[Parallel(n_jobs=-1)]: Done  49 out of  49 | elapsed:    4.3s finished

[2022-06-06 19:31:51] Features: 48/40 -- score: 0.876923076923077[Parallel(n_jobs=-1)]: Using backend LokyBackend with 80 concurrent workers.
[Parallel(n_jobs=-1)]: Done  14 out of  48 | elapsed:    1.8s remaining:    4.3s
[Parallel(n_jobs=-1)]: Done  39 out of  48 | elapsed:    3.8s remaining:    0.9s
[Parallel(n_jobs=-1)]: Done  48 out of  48 | elapsed:    4.3s finished

[2022-06-06 1

CPU times: user 35min 52s, sys: 29.8 s, total: 36min 22s
Wall time: 46min 41s


[Parallel(n_jobs=-1)]: Done  41 out of  41 | elapsed:    3.7s finished

[2022-06-06 19:32:24] Features: 40/40 -- score: 0.8846153846153847

In [9]:
feat_names = list(sfs1.k_feature_names_)
print("Fitur yang diambil adalah: ")
print(feat_names)
len(feat_names)

Fitur yang diambil adalah: 
['1138.2', '1138.7', '1139.2', '1140.7', '1132.2', '1131.7', '1134.2', '1360.3', '1361', '1131.2', '1272.8', '1358.2', '1262.3', '1270.9', '1273.4', '1266', '1274.7', '1369.6', '1269.7', '1279.7', '1660.9', '1279.1', '1349.7', '1651.4', '1657.8', '1000.3', '1353.9', '1278.5', '1130.2', '1673.8', '1298.3', '1258.6', '1701.3', '1178.5', '1681.4', '1318.1', '1321.5', '1364.6', '1688', '1258']


40

# Klasifikasi Random Forest Classifier

In [10]:
#buat n-fold cv
#cross validation 10-fold
cv = KFold(n_splits=10, random_state=1, shuffle=True)

## Testing Performa Model Random Forest Classifier

In [11]:
%%time
#tentukan metode scoring yang digunakan
scoring_rfe = {'acc': 'accuracy',
               'prec_micro': 'precision_micro',
               'rec_micro': 'recall_micro'}

#tentukan total fitur dan trees yang digunakan dalam proses klasifikasi ini
#n_feat = [10, 20, 30]
n_feat = [40]
n_trees = [100]

for nfeat in n_feat:
    for ntrees in n_trees:        
        #ambil n fitur input hasil seleksi fitur Forward 
        x_train_selected = x_train[feat_names].iloc[:,0:nfeat]
        x_test_selected = x_test[feat_names].iloc[:,0:nfeat]

        #Create a Gaussian Classifier
        clf_rfe = RandomForestClassifier(n_estimators=ntrees)
        
        #Train the model using the training sets
        clf_rfe.fit(x_train_selected, y_train)
        y_pred_rfe=clf_rfe.predict(x_test_selected)
        
        #hitung score model dari data train
        scores_rfe = cross_validate(clf_rfe, x_train_selected, y_train, scoring=scoring_rfe, cv=cv, return_train_score=True)
        
        print("Akurasi model RFE data Train dengan " + str(nfeat) + " fitur dan " + str(ntrees) + " trees:" 
              + str(round(scores_rfe['train_acc'].mean(), 2)))
        print("Akurasi model RFE data Test dengan " + str(nfeat) + " fitur dan " + str(ntrees) + " trees:" 
              + str(round(metrics.accuracy_score(y_test, y_pred_rfe), 2)))
        print("Precision model RFE data Train dengan " + str(nfeat) + " fitur dan " + str(ntrees) + " trees:" 
              + str(round(scores_rfe['train_prec_micro'].mean(), 2)))
        print("Precision model RFE data Test dengan " + str(nfeat) + " fitur dan " + str(ntrees) + " trees:" 
              + str(round(metrics.precision_score(y_test, y_pred_rfe, average='micro'), 2)))
        print("Recall model RFE data Train dengan " + str(nfeat) + " fitur dan " + str(ntrees) + " trees:" 
              + str(round(scores_rfe['train_rec_micro'].mean(), 2)))
        print("Recall model RFE data Test dengan " + str(nfeat) + " fitur dan " + str(ntrees) + " trees:" 
              + str(round(metrics.recall_score(y_test, y_pred_rfe, average='micro'), 2)))
        print(" ")

Akurasi model RFE data Train dengan 40 fitur dan 100 trees:1.0
Akurasi model RFE data Test dengan 40 fitur dan 100 trees:0.95
Precision model RFE data Train dengan 40 fitur dan 100 trees:1.0
Precision model RFE data Test dengan 40 fitur dan 100 trees:0.95
Recall model RFE data Train dengan 40 fitur dan 100 trees:1.0
Recall model RFE data Test dengan 40 fitur dan 100 trees:0.95
 
CPU times: user 1.34 s, sys: 16 ms, total: 1.36 s
Wall time: 1.36 s


In [12]:
%%time
#tentukan metode scoring yang digunakan
scoring_rfe = {'acc': 'accuracy',
               'prec_micro': 'precision_micro',
               'rec_micro': 'recall_micro'}

#tentukan total fitur dan trees yang digunakan dalam proses klasifikasi ini
#n_feat = [10, 20, 30]
n_feat = [40]
n_trees = [150]

for nfeat in n_feat:
    for ntrees in n_trees:        
        #ambil n fitur input hasil seleksi fitur Forward 
        x_train_selected = x_train[feat_names].iloc[:,0:nfeat]
        x_test_selected = x_test[feat_names].iloc[:,0:nfeat]

        #Create a Gaussian Classifier
        clf_rfe = RandomForestClassifier(n_estimators=ntrees)
        
        #Train the model using the training sets
        clf_rfe.fit(x_train_selected, y_train)
        y_pred_rfe=clf_rfe.predict(x_test_selected)
        
        #hitung score model dari data train
        scores_rfe = cross_validate(clf_rfe, x_train_selected, y_train, scoring=scoring_rfe, cv=cv, return_train_score=True)
        
        print("Akurasi model RFE data Train dengan " + str(nfeat) + " fitur dan " + str(ntrees) + " trees:" 
              + str(round(scores_rfe['train_acc'].mean(), 2)))
        print("Akurasi model RFE data Test dengan " + str(nfeat) + " fitur dan " + str(ntrees) + " trees:" 
              + str(round(metrics.accuracy_score(y_test, y_pred_rfe), 2)))
        print("Precision model RFE data Train dengan " + str(nfeat) + " fitur dan " + str(ntrees) + " trees:" 
              + str(round(scores_rfe['train_prec_micro'].mean(), 2)))
        print("Precision model RFE data Test dengan " + str(nfeat) + " fitur dan " + str(ntrees) + " trees:" 
              + str(round(metrics.precision_score(y_test, y_pred_rfe, average='micro'), 2)))
        print("Recall model RFE data Train dengan " + str(nfeat) + " fitur dan " + str(ntrees) + " trees:" 
              + str(round(scores_rfe['train_rec_micro'].mean(), 2)))
        print("Recall model RFE data Test dengan " + str(nfeat) + " fitur dan " + str(ntrees) + " trees:" 
              + str(round(metrics.recall_score(y_test, y_pred_rfe, average='micro'), 2)))
        print(" ")

Akurasi model RFE data Train dengan 40 fitur dan 150 trees:1.0
Akurasi model RFE data Test dengan 40 fitur dan 150 trees:0.95
Precision model RFE data Train dengan 40 fitur dan 150 trees:1.0
Precision model RFE data Test dengan 40 fitur dan 150 trees:0.95
Recall model RFE data Train dengan 40 fitur dan 150 trees:1.0
Recall model RFE data Test dengan 40 fitur dan 150 trees:0.95
 
CPU times: user 1.98 s, sys: 20.1 ms, total: 2 s
Wall time: 2 s


In [13]:
%%time
#tentukan metode scoring yang digunakan
scoring_rfe = {'acc': 'accuracy',
               'prec_micro': 'precision_micro',
               'rec_micro': 'recall_micro'}

#tentukan total fitur dan trees yang digunakan dalam proses klasifikasi ini
#n_feat = [10, 20, 30]
n_feat = [40]
n_trees = [200]

for nfeat in n_feat:
    for ntrees in n_trees:        
        #ambil n fitur input hasil seleksi fitur Forward 
        x_train_selected = x_train[feat_names].iloc[:,0:nfeat]
        x_test_selected = x_test[feat_names].iloc[:,0:nfeat]

        #Create a Gaussian Classifier
        clf_rfe = RandomForestClassifier(n_estimators=ntrees)
        
        #Train the model using the training sets
        clf_rfe.fit(x_train_selected, y_train)
        y_pred_rfe=clf_rfe.predict(x_test_selected)
        
        #hitung score model dari data train
        scores_rfe = cross_validate(clf_rfe, x_train_selected, y_train, scoring=scoring_rfe, cv=cv, return_train_score=True)
        
        print("Akurasi model RFE data Train dengan " + str(nfeat) + " fitur dan " + str(ntrees) + " trees:" 
              + str(round(scores_rfe['train_acc'].mean(), 2)))
        print("Akurasi model RFE data Test dengan " + str(nfeat) + " fitur dan " + str(ntrees) + " trees:" 
              + str(round(metrics.accuracy_score(y_test, y_pred_rfe), 2)))
        print("Precision model RFE data Train dengan " + str(nfeat) + " fitur dan " + str(ntrees) + " trees:" 
              + str(round(scores_rfe['train_prec_micro'].mean(), 2)))
        print("Precision model RFE data Test dengan " + str(nfeat) + " fitur dan " + str(ntrees) + " trees:" 
              + str(round(metrics.precision_score(y_test, y_pred_rfe, average='micro'), 2)))
        print("Recall model RFE data Train dengan " + str(nfeat) + " fitur dan " + str(ntrees) + " trees:" 
              + str(round(scores_rfe['train_rec_micro'].mean(), 2)))
        print("Recall model RFE data Test dengan " + str(nfeat) + " fitur dan " + str(ntrees) + " trees:" 
              + str(round(metrics.recall_score(y_test, y_pred_rfe, average='micro'), 2)))
        print(" ")

Akurasi model RFE data Train dengan 40 fitur dan 200 trees:1.0
Akurasi model RFE data Test dengan 40 fitur dan 200 trees:0.95
Precision model RFE data Train dengan 40 fitur dan 200 trees:1.0
Precision model RFE data Test dengan 40 fitur dan 200 trees:0.95
Recall model RFE data Train dengan 40 fitur dan 200 trees:1.0
Recall model RFE data Test dengan 40 fitur dan 200 trees:0.95
 
CPU times: user 2.62 s, sys: 28.1 ms, total: 2.65 s
Wall time: 2.64 s


# Klasifikasi Support Vector Machine

Untuk train dan test set, serta cv-nya ambil dari yang Random Forest Classifier, sama aja 

## Testing Performa Model Support Vector Machine

In [14]:
%%time
#tentukan metode scoring yang digunakan
scoring_svm = {'acc': 'accuracy',
               'prec_micro': 'precision_micro',
               'rec_micro': 'recall_micro'}

#tentukan total fitur yang digunakan dalam proses klasifikasi SVM ini
#n_feat = [10, 20, 30]
n_feat = [40]

for nfeat in n_feat:     
    #ambil n fitur input hasil seleksi fitur Forward 
    x_train_selected = x_train[feat_names].iloc[:,0:nfeat]
    x_test_selected = x_test[feat_names].iloc[:,0:nfeat]

    #Create a Support Vector Classifier
    clf_svm = svm.SVC()

    #Train the model using the training sets
    clf_svm.fit(x_train_selected, y_train)
    y_pred_svm = clf_svm.predict(x_test_selected)

    #hitung score model dari data train
    scores_svm = cross_validate(clf_svm, x_train_selected, y_train, scoring=scoring_svm, cv=cv, return_train_score=True)

    print("akurasi model SVM data Train dengan " + str(nfeat) + " fitur: " 
          + str(round(scores_svm['train_acc'].mean(), 2)))
    print("akurasi model SVM data Test dengan " + str(nfeat) + " fitur: " 
          + str(round(metrics.accuracy_score(y_test, y_pred_svm), 2)))
    print("Precision model SVM data Train dengan " + str(nfeat) + " fitur:"
          + str(round(scores_svm['train_prec_micro'].mean(), 2)))
    print("Precision model SVM data Test dengan " + str(nfeat) + " fitur:"
          + str(round(metrics.precision_score(y_test, y_pred_svm, average='micro'), 2)))
    print("Recall model SVM data Train dengan " + str(nfeat) + " fitur:"
          + str(round(scores_svm['train_rec_micro'].mean(), 2)))
    print("Recall model SVM data Test dengan " + str(nfeat) + " fitur:"
          + str(round(metrics.recall_score(y_test, y_pred_svm, average='micro'), 2)))
    print(" ")

akurasi model SVM data Train dengan 40 fitur: 0.74
akurasi model SVM data Test dengan 40 fitur: 0.62
Precision model SVM data Train dengan 40 fitur:0.74
Precision model SVM data Test dengan 40 fitur:0.62
Recall model SVM data Train dengan 40 fitur:0.74
Recall model SVM data Test dengan 40 fitur:0.62
 
CPU times: user 116 ms, sys: 3.92 ms, total: 120 ms
Wall time: 116 ms
